Things to do:
1. [x] The get_contibution methods currently use a mix of predict_proba and predict. This should be fixed - expect probs, and calculate from there.
2. [x] Compare the runtime of get_contributions and get_contributions_slow. Check on long signal windows. If the fast version isn't faster, use the slow version (its more user friendly). **the faster script is faster, and confirmed to work**
3. [ ] Try this tutorial script with a multivariate dataset
4. [ ] Try this tutorial with unequal lengths (like japanese vowels)
5. [ ] Try this tutorial with a regression model
6. [ ] Try this tutorial with a forecasting model

Multivariate options:
https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.datasets.load_basic_motions.html#sktime.datasets.load_basic_motions
https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.datasets.load_japanese_vowels.html#sktime.datasets.load_japanese_vowels

Leaving off notes:
- To make predictions with sktime models without having to do any formatting, call predict on X_train_multi.iloc[0:1] (or a range of rows, even if just one). This works for univariate and multivariate.
- Check the cell below with a bunch of predictions to see that the current occlusion implementation has an extra index, and the code works if we don't convert to numpy

## Univariate Timeseries Classification

In [ ]:
from sktime.datasets import load_arrow_head  # univariate dataset
X_train_orig, y_train_orig = load_arrow_head(split="train", return_X_y=True)
X_test_orig, y_test_orig = load_arrow_head(split="test", return_X_y=True)

X_train_orig

In [2]:
from sktime.classification.shapelet_based import ShapeletTransformClassifier

model = ShapeletTransformClassifier()
model.fit(X_train_orig, y_train_orig)

model.score(X_test_orig, y_test_orig)

0.7542857142857143

In [3]:
import importlib
import pyreal.explainers.time_series.saliency.occlusion_saliency as oi
importlib.reload(oi)
import time
from pyreal.transformers import Transformer
from sktime.datatypes._panel._convert import from_nested_to_3d_numpy

#class NestedToNumpy2D(Transformer):
#    def __transform__(x):
#        return x.to_frame().T

class DfToNumpy(Transformer):
    def data_transform(self,x):
        return from_nested_to_3d_numpy(x)

transformers = [DfToNumpy()]

model.predict = model.predict_proba
explainer = oi.OcclusionSaliency(model, X_train_orig, transformers=transformers, window_size=3)
explainer.fit()

start = time.time()
explanation1 = explainer.produce(X_train_orig.iloc[0])
explanation2 = explainer.produce(X_train_orig.iloc[1])

getting sklearn classes
(1, 1, 251)
(1, 1, 251)


In [4]:
import pyreal.utils.visualize as vis
importlib.reload(vis)

explanation1[0].shape
vis.plot_timeseries_saliency(X_train_orig.iloc[0][0], explanation1[0][0:245])
#vis.plot_timeseries_saliency(X_train_orig.iloc[1][0], explanation2[0][0:245])

ValueError: Collections can only map rank 1 arrays

<Figure size 432x288 with 2 Axes>

## Multivariate Timeseries Classification

In [5]:
from sktime.datasets import load_basic_motions #multivariate

X_train_multi, y_train_multi = load_basic_motions(split="train", return_X_y=True)
X_test_multi, y_test_multi = load_basic_motions(split="test", return_X_y=True)

In [6]:
from sktime.classification.shapelet_based import ShapeletTransformClassifier

model_multi = ShapeletTransformClassifier()
model_multi.fit(X_train_multi, y_train_multi)

model_multi.score(X_test_multi, y_test_multi)

1.0

In [ ]:
import importlib
import pyreal.explainers.time_series.saliency.occlusion_saliency as oi
importlib.reload(oi)
import time

transformers = [DfToNumpy()]

model_multi.predict = model_multi.predict_proba
explainer = oi.OcclusionSaliency(model_multi, X_train_multi, transformers=transformers, window_size=3)
explainer.fit()

start = time.time()
explanation_multi1 = explainer.produce(X_train_multi.iloc[0])
explanation_multi2 = explainer.produce(X_train_multi.iloc[1])

getting sklearn classes
(1, 6, 100)


In [ ]:
explanation2[0].shape
X_train_multi.shape
#for f in range(X_train_multi.shape[1]):
#    vis.plot_timeseries_saliency(X_train_multi.iloc[0][f], explanation1[0][0:245])

In [ ]:
X_train_multi.iloc[0].__class__

In [ ]:
X_train_orig.iloc[0]

In [ ]:
import numpy as np
from sktime.datatypes._panel._convert import from_nested_to_2d_array

sig = np.copy(X_train_orig.iloc[0][0].to_frame().T)

print(model.predict(sig))
print(model.predict(X_train_orig.iloc[0:1]))
print(model.predict(X_train_orig.iloc[0].to_frame().T))
print(model.predict(from_nested_to_3d_numpy(X_train_orig.iloc[0].to_frame().T)))
#np.argmax(model.predict(sig))

print(X_train_orig.iloc[0])

#print(sig.shape)
#print("---")
#print(X_train_orig.iloc[0].to_frame().T.shape)

In [ ]:
from sktime.datatypes._panel._convert import from_nested_to_3d_numpy

sig = np.copy(X_train_orig.iloc[0][0].to_frame().T)

print(model_multi.predict(X_train_multi.iloc[0:1]))
print(model_multi.predict(X_train_multi.iloc[0].to_frame().T))
print(model_multi.predict(from_nested_to_3d_numpy(X_train_multi.iloc[0:1])))
print(model.predict(from_nested_to_3d_numpy(X_train_multi.iloc[0].to_frame().T)))

print(model.predict(from_nested_to_3d_numpy(X_train_multi.iloc[0].to_frame().T))[0])

#print(X_train_multi.iloc[0].to_frame().T)
#print(np.copy(X_train_multi.iloc[0].to_frame().T))
#print(from_nested_to_3d_numpy(X_train_multi.iloc[0:1]))

In [ ]:
sig

## Multivariate Timeseries Classification (Variable Length)

In [ ]:
from sktime.datasets import load_japanese_vowels #multivariate variable length

X_train_multi, y_train_multi = load_japanese_vowels(split="train", return_X_y=True)
X_test_multi, y_test_multi = load_japanese_vowels(split="test", return_X_y=True)